In [2]:
import os
import keras
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [13]:
# train data
train_dir = 'data/train'
#validation data
val_dir = 'data/val'
#test data
img_for_test_dir='/data/test'
# images shape
img_width, img_height = 224, 224
# tensor shape
input_shape = (img_width, img_height, 3)
# number epoch
epochs = 40
# mini-batch size
batch_size = 16

In [4]:
model_resnet50=keras.applications.resnet50.ResNet50(include_top=False,
                                               weights="imagenet",
                                               input_shape=input_shape)

W1004 15:41:13.306985 11936 deprecation_wrapper.py:119] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1004 15:41:15.287275 11936 deprecation_wrapper.py:119] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1004 15:41:16.193594 11936 deprecation_wrapper.py:119] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1004 15:41:16.553742 11936 deprecation_wrapper.py:119] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_ses

In [7]:
# add a global spatial average pooling layer
x = model_resnet50.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=model_resnet50.input, outputs=predictions)

In [8]:
optimizer=Adam(lr=0.00001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


W1004 15:44:37.066808 11936 deprecation_wrapper.py:119] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [14]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                                  


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=100,
    class_mode='categorical')

Found 22424 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [15]:
reducelronplateau=ReduceLROnPlateau(monitor='val_loss',         #callback which reduces the speed of training if the losses do not decrease
                                    factor=0.1,
                                    patience=10)

callbacks=[ModelCheckpoint("saved_models/drivedetvgg16-dense-{epoch:02d}-{val_acc:.4f}.hdf5"),reducelronplateau]

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 100,
    callbacks=callbacks)


W1004 15:52:46.759339 11936 deprecation.py:323] From C:\Users\Elena\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
